### Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import random # library for random number generation

!pip install geopy
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from IPython.display import Image 
from IPython.core.display import HTML 

!conda install --override-channels -c main package_in_main

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - package_in_main

Current channels:

  - https://conda.anaconda.org/main/linux-64
  - https://conda.anaconda.org/main/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported.


### Foursquare data

In [2]:
CLIENT_ID = '0ZLRMIPIQ1MNB4IROWDSKAIZTRD3JAGVOTHDUR3YTCIGVKXS' # your Foursquare ID
CLIENT_SECRET = 'BWJHQAANUAO1AVG45F4LZUIL3MCJ1CTKMAMUNMIMUTT5AWOA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ZLRMIPIQ1MNB4IROWDSKAIZTRD3JAGVOTHDUR3YTCIGVKXS
CLIENT_SECRET:BWJHQAANUAO1AVG45F4LZUIL3MCJ1CTKMAMUNMIMUTT5AWOA


### Define Coordinares of the Center location

In [3]:
geolocator = Nominatim (user_agent="foursquare_agent")
latitude = 52.4454164
longitude = 13.5752346
categoryId = '52f2ab2ebcbc57f1066b8b46'
radius = 2000
print(latitude, longitude)

52.4454164 13.5752346


### Define the URL 

In [4]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&radius={}&v=20201226'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    latitude, 
    longitude,
    categoryId, 
    radius)
results = requests.get(url).json()

### Transform the Data into a Pandas Dataframe 

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
                   
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
venues = results["response"]['venues']
#venues = results["response"]['groups'][0]['items']
# flatten JSON to venues
nearby_venues = json_normalize(venues)


# keep only columns that include venue name, and anything that is associated with location 
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
nearby_venues1 = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]
nearby_venues1.drop(columns=['labeledLatLngs','categories', 'cc', 'city', 'state', 'country', 'formattedAddress', 'id', 'crossStreet', 'neighborhood'], inplace = True)


nearby_venues1

,name,address,lat,lng,distance,postalCode
0,ALDI NORD,Müggelheimer Str. 36,52.444132,13.585254,694,12555
1,REWE,Oberspreestraße 145,52.446150,13.564179,754,12555
2,NORMA,Seelenbinderstr. 112-120,52.454878,13.590302,1467,12555
3,PENNY,Wendenschloßstraße 154,52.437592,13.580990,954,12557
4,EDEKA Wendenschloss,Wendenschloßstraße 324,52.428188,13.584579,2019,12557
5,Lidl,Adlergestell 291,52.431322,13.548144,2416,12489
6,REWE,Pablo-Neruda-Straße 2,52.441346,13.587504,947,12559
7,REWE,Bahnhofstraße 33-38,52.457862,13.576878,1389,12555
8,REWE,Wendenschloßstraße 354-358,52.425246,13.585077,2342,12557
9,Lidl,Wendenschloßstr. 9,52.446815,13.593208,1229,12559


12459, 12555, 12557, 12559, 12587

In [7]:
print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))

21 venues were returned by Foursquare.


### Create the Map

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
# generate map centred around the Rathaus in Koepenick
# add a red circle marker to represent the central point - Rathaus in Koepenick
folium.CircleMarker(
    [latitude, longitude],
    radius=7,
    color='red',
    popup='Rathaus in Koepenick',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6).add_to(venues_map)

# add the supermarkets as blue circle markers
for lat, lng, label in zip(nearby_venues1.lat, nearby_venues1.lng, nearby_venues1.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=folium.Popup(label, parse_html=True),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6).add_to(venues_map)
   

display(venues_map)
            

### K-means

In [9]:
lockers_grouped = nearby_venues1.groupby('distance').mean().reset_index()
lockers_grouped_clustering = lockers_grouped.drop('distance', 1)

In [10]:
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lockers_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:21] 
lockers_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
lockers_grouped_clustering

,Cluster Labels,lat,lng
0,3,52.444132,13.585254
1,4,52.446150,13.564179
2,3,52.441346,13.587504
3,3,52.437592,13.580990
4,4,52.436744,13.570502
5,0,52.452205,13.586725
6,3,52.446815,13.593208
7,0,52.455799,13.586381
8,0,52.457652,13.578826
9,0,52.457862,13.576878


In [11]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lockers_grouped['lat'], lockers_grouped['lng'], lockers_grouped['distance'], lockers_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters